In [1]:
import sys
import pickle
import pandas as pd
import glob
import re

In [ ]:
# TODO define the folder of etabs you want to study here
STUDY_PATH = "../../frappier_termless/tic0/*.etab" 

In [2]:
myAmino = ["R","H","K","D","E","S","T","N","Q","C","G","P","A","V","I","L","M","F","Y","W"]
FullAmino = ["ARG","HIS","LYS","ASP","GLU","SER","THR","ASN","GLN","CYS","GLY","PRO","ALA","VAL","ILE","LEU","MET","PHE","TYR","TRP"]
aminos = {FullAmino[i]:myAmino[i] for i in range(len(myAmino))}

In [3]:
#Benchmark dataset from SORTCERY
AllName = ["x1","m1","f100"]
#New name
NameTest = ["B2CL1_SORTCERY","MCL1_SORTCERY","B2LA1_SORTCERY"]

ApoDTerm = False

In [4]:
#Dictionnary of energy (exp or predicted)
PepBindEner = dict()

#Get Energy from sortery
for (t,name) in zip(AllName,NameTest):
    subdf = pd.read_csv("../SORTCERY_data/"+t+"_merged.csv")
    
    if name not in PepBindEner:
        PepBindEner[name] = dict()
    for (seq,val) in zip(subdf["protein"],subdf[t+"_mean_ener"]):
        if seq in PepBindEner[name]:
            print(name,seq,val,PepBindEner[name][seq])
        PepBindEner[name][seq] = float(val)
    print(t,name,len(subdf),len(PepBindEner[name]))
    
    
#Load Binary
for (t,n) in zip(AllName,NameTest):
    subdf = pd.read_csv("../SORTCERY_data/"+t+"_binary_merged.csv")
    name = "binary_"+n
    if name not in PepBindEner:
        PepBindEner[name] = dict()
    for (seq,val) in zip(subdf["protein"],subdf["binder"]):
        if seq in PepBindEner[name]:
            continue
            print(name,seq,val,PepBindEner[name][seq])
        PepBindEner[name][seq] = float(val)
    print(t,name,len(subdf),len(PepBindEner[name]))
    #100-80*subdf["binder"]

x1 B2CL1_SORTCERY 4386 4386
m1 MCL1_SORTCERY 4491 4491
f100 B2LA1_SORTCERY 3805 3805
x1 binary_B2CL1_SORTCERY 1882 1872
m1 binary_MCL1_SORTCERY 1940 1855
f100 binary_B2LA1_SORTCERY 1686 1686


In [5]:
#Load Bcl-2 Protoeme screen
Bcl2Prot = pd.read_table("../SORTCERY_data/bcl2_proteome_screen.dat")
for (uni,sname) in zip(["B2CL1","MCL1","B2LA1"],["Bcl-xL","Mcl-1","Bfl-1"]):
    print(uni,sname)
    name = "binary_"+uni+"_Proteome"
    if name not in PepBindEner:
        PepBindEner[name] = dict()
    subdf = Bcl2Prot.dropna(subset=[sname])
    for (seq,val) in zip(subdf["Peptide"],subdf[sname]):
        nseq = "AA"+seq
        PepBindEner[name][nseq] = float(val=="binder")

B2CL1 Bcl-xL
MCL1 Mcl-1
B2LA1 Bfl-1


In [6]:
#Get Score from FoldX and Rosetta
AllEnergyRos = pickle.load( open( "../computed_data/RosFoldEner.pickle", "rb" ))

In [7]:
for mod in AllEnergyRos.keys():
    if re.search("_Rosetta$",mod):
        continue
    #Make new model name so last term is the model name
    RegMatch = re.search("^(\S+_holo_)(\S+)$",mod)
    nmod = RegMatch.group(1)+"".join(RegMatch.group(2).split("_"))
    
    #Init dict
    if mod not in PepBindEner:
        PepBindEner[nmod] = dict()
    
    for seq in AllEnergyRos[mod]:
        if None == AllEnergyRos[mod][seq]:
            print(seq,mod)
            continue
        if not re.search("\d+",str(AllEnergyRos[mod][seq])):
            print(seq,mod,AllEnergyRos[mod][seq])
            continue
        PepBindEner[nmod][seq] = float(AllEnergyRos[mod][seq])

In [8]:
#Score sequence with dTERMen

In [9]:
def load_dTERMen_sc(etab):
    #Initialise and read energy table        
    allpos = []
    lines = [line.rstrip('\n') for line in open(etab)]
    holo = dict()
    AllSelf = []
    # last = ""
    for l in lines:
        #print(l)
        sp = re.split("\s+",l)
        allpos.append(sp[0])
        
        if len(sp) == 3:
            AllSelf.append(float(sp[2]))
            # k1 = sp[0]+aminos[sp[1]]
            k1 = sp[0]+sp[1]
            if k1 not in holo:
                holo[k1] = dict()
                # if sp[0] != last:
                #     last = sp[0]
            holo[k1][k1] = float(sp[2])
            continue
        if len(sp) == 5:
            # k1 = sp[0]+aminos[sp[2]]
            # k2 = sp[1]+aminos[sp[3]]
            k1 = sp[0]+sp[2]
            k2 = sp[1]+sp[3]
            holo[k1][k2] = float(sp[4])
            AllSelf.append(float(sp[4]))
            continue
        print(l)
        break
        
    #Keep only unique position
    allpos = sorted(list(set(allpos)), key=lambda x: (x[0], int(x.split(",")[1]))) # sort by chain, then by index
    return(holo,allpos)

#Score seq using energy function
def score_sec(seq,mat,ref,chain_size=20):
    sc = 0.0
    for i in range(-1, -(chain_size+1), -1):
        k2 = ref[i]+seq[i]
        for j in range(i, -(len(seq)+1), -1):
            k1 = ref[j]+seq[j]
            if k2 not in mat[k1]:
                continue
            sc += mat[k1][k2]
    return sc

In [10]:
def get_full_native(model):
    pdb = "../pdb/clean_pdb/{}.pdb".format(model[:model.index("_35")])
    lines = [line.rstrip('\n') for line in open(pdb)]

    seq = []
    index = ""
    for l in lines:
        sp = re.split("\s+",l)
        if sp[0] != "ATOM":
            continue
        if sp[5] != index:
            index = sp[5]
            seq.append(aminos[sp[3]])
    
    return "".join(seq)

In [11]:
# testing
print(get_full_native("B2CL1_HUMAN_1BXL_A_holo_35"))

MSQSNRELVVDFLSYKLSQKGYSWSQFSDVEENRTEAPEGTESEAVKQALREAGDEFELRYRRAFSDLTSQLHITPGTAYQSFEQVVNELFRDGVNWGRIVAFFSFGGALCVESVDKEMQVLVSRIAAWMATYLNDHLEPWIQENGGWDTFVELYGNNAAAESRKGQERLGQVGRQLAIIGDDINR


In [12]:
ApoDTerm = False
CHAIN_LENGTH = 20
alletab = glob.glob(STUDY_PATH)
for etab in alletab:
    #Get Model name
    model = re.search("([A-Za-z0-9_]+)\.etab",etab).group(1)
    if len(model.split("_")) != 6:
        continue
    #Get receptor
    rec = model.split("_")[0]
    
    if "BCL2" in model:
        continue
        
    #Only human
    if "HUMAN" not in model:
        continue
    #Split Model
    sp = model.split("_")
    if len(sp) != 6:
        continue
    
    scoring = "dTERMenC"
    if "apo" in model:
        if ApoDTerm == False:continue
        scoring = "dTERMenA"
    sp[-1] = scoring+sp[-1]
    #Get new name
    nmodel = "_".join(sp)
    
    #Load scoring matrix
    (holo,allpos) = load_dTERMen_sc(etab)

    i = 1
    while allpos[-i][0] == "B":
        i += 1
    if i != CHAIN_LENGTH+1: # check for length 20 B chain
        print(model,len(allpos))
        continue

    print("now scoring:", model)
    # obtain native sequence of protein
    native_seq = get_full_native(model)[:-CHAIN_LENGTH]
    
    if nmodel not in PepBindEner:
        PepBindEner[nmodel] = dict()
    for seq in PepBindEner[rec+"_SORTCERY"]:
        #dTERMen pep are shorter, so need to adjust
        SeqToScore = str(seq)[2:CHAIN_LENGTH+2]
        #Score Seq
        ener = score_sec(native_seq+SeqToScore,holo,allpos)
        PepBindEner[nmodel][seq] = ener
        
    for seq in PepBindEner["binary_"+rec+"_SORTCERY"]:
        #dTERMen pep are shorter, so need to adjust
        SeqToScore = str(seq)[2:CHAIN_LENGTH+2]
        #Score Seq
        ener = score_sec(native_seq+SeqToScore,holo,allpos)
        PepBindEner[nmodel][seq] = ener
    for seq in PepBindEner["binary_"+rec+"_Proteome"]:
        #dTERMen pep are shorter, so need to adjust
        SeqToScore = str(seq)[2:CHAIN_LENGTH+2]
        #Score Seq
        ener = score_sec(native_seq+SeqToScore,holo,allpos)
        PepBindEner[nmodel][seq] = ener
    

B2CL1_HUMAN_1BXL_A_holo_35 186
now scoring: B2CL1_HUMAN_1G5J_A_holo_35
now scoring: B2CL1_HUMAN_2M04_A_holo_35
now scoring: B2CL1_HUMAN_2P1L_A_holo_35
now scoring: B2CL1_HUMAN_2P1L_C_holo_35
now scoring: B2CL1_HUMAN_2P1L_E_holo_35
now scoring: B2CL1_HUMAN_2P1L_G_holo_35
now scoring: B2CL1_HUMAN_2PON_B_holo_35
B2CL1_HUMAN_2YQ7_A_holo_35 157
now scoring: B2CL1_HUMAN_3FDL_A_holo_35
now scoring: B2CL1_HUMAN_3IO8_A_holo_35
B2CL1_HUMAN_3IO8_C_holo_35 159
now scoring: B2CL1_HUMAN_3PL7_A_holo_35
B2CL1_HUMAN_3R85_A_holo_35 157
B2CL1_HUMAN_3R85_B_holo_35 155
B2CL1_HUMAN_3R85_C_holo_35 157
B2CL1_HUMAN_3R85_D_holo_35 156
now scoring: B2CL1_HUMAN_4CIN_A_holo_35
B2CL1_HUMAN_4CIN_B_holo_35 153
now scoring: B2CL1_HUMAN_4HNJ_A_holo_35
now scoring: B2CL1_HUMAN_4QVE_A_holo_35
now scoring: B2CL1_HUMAN_4QVF_A_holo_35
B2CL1_HUMAN_5B1Z_A_holo_35 160
B2CL1_HUMAN_5B1Z_B_holo_35 161
B2CL1_HUMAN_5FMJ_A_holo_35 171
now scoring: B2CL1_HUMAN_5FMK_A_holo_35
now scoring: B2LA1_HUMAN_2VM6_A_holo_35
B2LA1_HUMAN_3I1H_A_

In [13]:
#Find how often each pdb is there
PDBCount = dict()
MaxPDB = 0
for k in sorted(PepBindEner.keys()):
    if len(PepBindEner[k]) < 1500:
        #print(k)
        continue
        die
    sp = k.split("_")
    if len(sp) < 6:
        continue
    pdb = "_".join(sp[0:4])
    if pdb not in PDBCount:
        PDBCount[pdb] = 0
    PDBCount[pdb] += 1
    if PDBCount[pdb] > MaxPDB:
        MaxPDB = PDBCount[pdb]
    #print(pdb,k,len(PepBindEner[k]))

#Remove incomplete template
for k in sorted(PepBindEner.keys()):
    if len(PepBindEner[k]) < 1500:
        #print(k)
        continue
    sp = k.split("_")
    if len(sp) < 6:
        continue
    pdb = "_".join(sp[0:4])
    if PDBCount[pdb] != MaxPDB:
        if k in PepBindEner:
            print(k,PDBCount[pdb],MaxPDB)
            del PepBindEner[k]
    

B2CL1_HUMAN_1BXL_A_holo_FoldX 7 8
B2CL1_HUMAN_1BXL_A_holo_RosettaTalarisInterfacedGcross 7 8
B2CL1_HUMAN_1BXL_A_holo_RosettaTalarisInterfacedGseparated 7 8
B2CL1_HUMAN_1BXL_A_holo_RosettaTalarisPose 7 8
B2CL1_HUMAN_1BXL_A_holo_RosettabetaInterfacedGcross 7 8
B2CL1_HUMAN_1BXL_A_holo_RosettabetaInterfacedGseparated 7 8
B2CL1_HUMAN_1BXL_A_holo_RosettabetaPose 7 8
B2CL1_HUMAN_2YQ7_A_holo_FoldX 7 8
B2CL1_HUMAN_2YQ7_A_holo_RosettaTalarisInterfacedGcross 7 8
B2CL1_HUMAN_2YQ7_A_holo_RosettaTalarisInterfacedGseparated 7 8
B2CL1_HUMAN_2YQ7_A_holo_RosettaTalarisPose 7 8
B2CL1_HUMAN_2YQ7_A_holo_RosettabetaInterfacedGcross 7 8
B2CL1_HUMAN_2YQ7_A_holo_RosettabetaInterfacedGseparated 7 8
B2CL1_HUMAN_2YQ7_A_holo_RosettabetaPose 7 8
B2CL1_HUMAN_3IO8_C_holo_FoldX 7 8
B2CL1_HUMAN_3IO8_C_holo_RosettaTalarisInterfacedGcross 7 8
B2CL1_HUMAN_3IO8_C_holo_RosettaTalarisInterfacedGseparated 7 8
B2CL1_HUMAN_3IO8_C_holo_RosettaTalarisPose 7 8
B2CL1_HUMAN_3IO8_C_holo_RosettabetaInterfacedGcross 7 8
B2CL1_HUMAN_3IO

In [14]:
#Get Min
#Find model

MinPDBTemp = dict()

for k in sorted(PepBindEner.keys()):
    sp = k.split("_")
    if len(sp) != 6:
        #print(k)
        continue
    
    mod = "_".join(sp[:2]+["MIN",sp[-1]])
    pdb = "_".join(k.split("_")[:4])
    
    if mod not in PepBindEner:
        PepBindEner[mod] = dict()
    #print(k,len(sp),mod)
    for seq in PepBindEner[k]:
        if seq not in PepBindEner[mod]:
            PepBindEner[mod][seq] = PepBindEner[k][seq]
        if mod not in MinPDBTemp:
            MinPDBTemp[mod] = dict()
        if PepBindEner[k][seq] <= PepBindEner[mod][seq]:
            PepBindEner[mod][seq] = PepBindEner[k][seq]
            
            MinPDBTemp[mod][seq] = pdb
#Get MinPDB count
MinPDB = dict()
for mod in MinPDBTemp.keys():
    submod = mod.split("_")[-1]
    if submod not in MinPDB:
        MinPDB[submod] = dict()
    for seq in MinPDBTemp[mod]:
        pdb = MinPDBTemp[mod][seq]
        if pdb not in MinPDB[submod]:
            MinPDB[submod][pdb] = 0
        MinPDB[submod][pdb] += 1

In [15]:
PepBindEnerKey = PepBindEner.keys()
for model in PepBindEnerKey:
    if ApoDTerm == False:continue
    if "dTERMenC" not in model:continue
    apo_model = re.sub("holo_dTERMenC","apo_dTERMenA",model)
    if "MIN" in model:
        apo_model = re.sub("dTERMenC","dTERMenA",model)
    if apo_model not in PepBindEner:
        print(model)
        continue
    dif_model = re.sub("dTERMenC","dTERMenD",model)
    PepBindEner[dif_model] = dict()
    for seq in PepBindEner[model]:
        PepBindEner[dif_model][seq] =  PepBindEner[model][seq] - PepBindEner[apo_model][seq]
        #print(seq,PepBindEner[model][seq],PepBindEner[apo_model][seq],PepBindEner[dif_model][seq])
    


In [16]:
PepBindEnerDF = pd.DataFrame(PepBindEner).dropna(axis=1, how='all').dropna(axis=0, how='all')

In [17]:
PepBindEnerDF.to_csv("../computed_data/PepBindEnerDF.csv")
pickle.dump(PepBindEnerDF,open("../computed_data/PepBindEnerDF.pickle","wb"))